In [ ]:
# Install necessary packages
# !pip install pennylane tqdm  # for google collab



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 44.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from google.colab import drive
from tqdm import tqdm


In [ ]:

# Mount Google Drive
drive.mount('/content/drive')

# Set data directory
DATA_DIR = "/content/drive/MyDrive/Stocks"



Mounted at /content/drive


In [ ]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Load data
all_data = []
for file in os.listdir(DATA_DIR):
    if file.endswith(".txt"):
        file_path = os.path.join(DATA_DIR, file)
        if os.path.getsize(file_path) == 0:
            print(f"Skipping empty file: {file}")
            continue
        try:
            df = pd.read_csv(file_path, sep=",", header=0, on_bad_lines="skip", encoding="utf-8")
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
            if df.isnull().all().all():
                print(f"Skipping invalid file: {file}")
                continue
            all_data.append(df)
            print(f"Loaded: {file}, Shape: {df.shape}")
        except Exception as e:
            print(f"Error loading {file}: {e}")


Streaming output truncated to the last 5000 lines.
Loaded: lmb.us.txt, Shape: (237, 7)
Loaded: ldf.us.txt, Shape: (3200, 7)
Loaded: kn.us.txt, Shape: (931, 7)
Loaded: rare.us.txt, Shape: (954, 7)
Loaded: maxr.us.txt, Shape: (392, 7)
Loaded: mcrb.us.txt, Shape: (601, 7)
Loaded: pkbk.us.txt, Shape: (2924, 7)
Loaded: mei.us.txt, Shape: (3201, 7)
Loaded: intg.us.txt, Shape: (2184, 7)
Loaded: mdc.us.txt, Shape: (3201, 7)
Loaded: sar.us.txt, Shape: (2653, 7)
Loaded: plm.us.txt, Shape: (3200, 7)
Loaded: lret.us.txt, Shape: (549, 7)
Loaded: lng.us.txt, Shape: (3201, 7)
Loaded: kim_k.us.txt, Shape: (1238, 7)
Loaded: rpt.us.txt, Shape: (3201, 7)
Loaded: ll.us.txt, Shape: (2520, 7)
Loaded: hfro.us.txt, Shape: (5, 7)
Loaded: nm.us.txt, Shape: (3199, 7)
Loaded: ix.us.txt, Shape: (3201, 7)
Loaded: prh.us.txt, Shape: (1172, 7)
Loaded: sfm.us.txt, Shape: (1080, 7)
Loaded: qgen.us.txt, Shape: (3201, 7)
Loaded: ktp.us.txt, Shape: (1816, 7)
Loaded: kemp.us.txt, Shape: (415, 7)
Loaded: iphs.us.txt, Shape:

In [ ]:
# Combine all data
df = pd.concat(all_data, ignore_index=True)

In [ ]:

# Feature selection
features = ["Close", "Open", "High", "Low", "Volume"]
target = "Close"
X = df[features]
y = df[target]

In [ ]:
# Normalize data
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [ ]:
# Split dataset
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
# DataLoader setup
batch_size = 1024
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(TensorDataset(X_valid_tensor, y_valid_tensor), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)


In [ ]:
# Define quantum device
dev = qml.device("default.qubit", wires=4)

def qbm_circuit(params):
    for i in range(4):
        qml.RY(params[i], wires=i)
        qml.RX(params[i+4], wires=i)
        qml.RZ(params[i+8], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

qnode = qml.QNode(qbm_circuit, dev, interface="torch")

class QuantumBoltzmannMachine(nn.Module):
    def __init__(self):
        super().__init__()
        self.params = nn.Parameter(0.01 * torch.randn(12, device=device))
        self.fc1 = nn.Linear(len(features) + 4, 128).to(device)
        self.fc2 = nn.Linear(128, 64).to(device)
        self.fc3 = nn.Linear(64, 1).to(device)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(64)

    def forward(self, x):
        q_output = qnode(self.params)
        q_output = torch.tensor(q_output, dtype=torch.float32, requires_grad=True, device=device) / 10
        x = torch.cat([x, q_output.repeat(x.size(0), 1)], dim=1)
        x = torch.relu(self.batch_norm1(self.fc1(x)))
        x = torch.relu(self.batch_norm2(self.fc2(x)))
        return self.fc3(x)

model = QuantumBoltzmannMachine().to(device)
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)


In [ ]:
num_epochs = 30
best_valid_loss = float("inf")
print("Training on", device)
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch) * 100
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")

Training on cpu


Epoch 1/30: 100%|██████████| 10178/10178 [04:36<00:00, 36.76it/s]


Epoch 1/30 - Train Loss: 1.3437


Epoch 2/30: 100%|██████████| 10178/10178 [04:39<00:00, 36.41it/s]


Epoch 2/30 - Train Loss: 1.0904


...

Epoch 28/30 - Train Loss: 1.5548


Epoch 29/30: 100%|██████████| 10178/10178 [05:01<00:00, 33.75it/s]


Epoch 29/30 - Train Loss: 1.5545


Epoch 30/30: 100%|██████████| 10178/10178 [04:59<00:00, 34.03it/s]

Epoch 30/30 - Train Loss: 1.0032


In [ ]:
# Save model
MODEL_PATH = "/content/drive/MyDrive/best_qbm_model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

Model saved to /content/drive/MyDrive/best_qbm_model.pth


In [ ]:
# Load and test model
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()
test_loss = 0.0
all_predictions, all_actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        test_loss += criterion(predictions, y_batch).item()
        all_predictions.extend(predictions.cpu().numpy())
        all_actuals.extend(y_batch.cpu().numpy())
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")


Test Loss: 0.0056


In [ ]:

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(all_actuals, label="Actual Prices", linestyle='dashed')
# plt.plot(all_predictions, label="Predicted Prices")
plt.legend()
plt.xlabel("Test Sample Index")
plt.ylabel("Stock Price")
plt.title("Quantum Boltzmann Machine: Predicted vs Actual Stock Prices")
plt.show()


NameError: name 'plt' is not defined